# Create, update, invoking KBs + LLaMa based RAG models & Evaluating responses with open source frameworks for a product use case: Medical Health

### Introduction

#### This guide outlines how to leverage Amazon Bedrock's knowledge base and Retrieval Augmented Generation (RAG) capabilities for a medical health product. The process involves configuring a RAG-based application to query a knowledge base, generating responses augmented with retrieved information. This technique enhances the capabilities of Large Language Models (LLMs) like LLaMa, enabling more informed and context-rich responses.

### Steps for Setup and Usage

1. Configure Data Sources: Determine the medical information to be included in your knowledge base.

2. Upload Data to Amazon S3: Transfer your prepared medical data to an S3 bucket for accessibility and storage.

3. Data Ingestion: Utilize a foundation model to generate embeddings of your data. Store these embeddings in a supported vector store.

4. Application/Agent Setup: Configure your application or agent to query the knowledge base, ensuring it returns responses augmented with the retrieved data.

### How It Works

#### 1. Retrieval Augmented Generation (RAG): Amazon Bedrock's knowledge base facilitates RAG by drawing information from data sources to supplement responses from LLMs.

#### 2. Pre-processing Data: Documents are chunked, converted to embeddings, and stored in a vector index with a link to the original document.

#### 3. Runtime Execution: User queries are converted to vectors, matched with document vectors from the knowledge base, and used to augment responses.


In [2]:
#install knowledge base sdk
%pip install --upgrade pip
%pip install boto3 --force-reinstall
%pip install botocore --force-reinstall
%pip install botocore --force-reinstall
%pip install langchain --force-reinstall --quiet

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 256.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 218.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 288.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 285.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: jmespath
    Found existing installation: jmespath 1.0

### Configure your bedrock client:

In [3]:
import boto3
import pprint
from botocore.client import Config
from langchain.llms.bedrock import Bedrock
from IPython.display import Markdown, display
from langchain.embeddings import BedrockEmbeddings

bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime')
agent_client = boto3.client('bedrock-agent')
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              region_name='us-east-1',
                              config=bedrock_config)

# we will be using the Titan Embeddings Model to generate our Embeddings.
embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-g1-text-02", client=bedrock_client)

## Creating the KB using the API for knowledge bases

In [23]:
role_arn = ''  # Replace with your Role ARN
embedding_model_arn = ''  # Corrected Embedding Model ARN

# Choose your vector store type and configure accordingly
# Example for Amazon OpenSearch Service
storage_configuration = {
    'opensearchServerlessConfiguration': {
        'collectionArn': '',  # Replace with your Collection ARN
        'fieldMapping': {
            'metadataField': '',  # Replace with your Metadata Field
            'textField': '',  # Replace with your Text Field
            'vectorField': ''  # Replace with your Vector Field
        },
        'vectorIndexName': ''  # Replace with your Vector Index Name
    },
    'type': ''  # Corrected type
}

# Creating the knowledge base
try:
    medical_KB = agent_client.create_knowledge_base(
        name='medical_KB', 
        description='KB that contains information on medicines and health',
        roleArn=role_arn,
        knowledgeBaseConfiguration={
            'type': 'VECTOR',  # Corrected type
            'vectorKnowledgeBaseConfiguration': {
                'embeddingModelArn': embedding_model_arn
            }
        },
        storageConfiguration=storage_configuration
    )

    # Pretty print the response
    pprint.pprint(medical_KB)

except Exception as e:
    print(f"Error occurred: {e}")


## Create a data source that you can attach to this KB:

In [24]:
# Define the S3 configuration for your data source
s3_configuration = {
    'bucketArn': 'arn:aws:s3:::medicaldata2039',  # Replace with the ARN of your S3 bucket
    'inclusionPrefixes': ['*']  # Assuming you want to include all files in the bucket
}

# Define the data source configuration
data_source_configuration = {
    's3Configuration': s3_configuration,
    'type': 'S3'  # Type of data source, in this case, S3
}

# Replace with your knowledge base ID
knowledge_base_id = ''

# Create the data source
try:
    data_source_response = agent_client.create_data_source(
        knowledgeBaseId=knowledge_base_id,
        name='MedicalDataSource',
        description='DataSource for medical data',
        dataSourceConfiguration=data_source_configuration
    )

    # Pretty print the response
    pprint.pprint(data_source_response)

except Exception as e:
    print(f"Error occurred: {e}")


Creating a data source in Amazon Bedrock involves configuring a connection to an external storage system where your data is hosted. In this case, the data is stored in an Amazon S3 bucket named medicaldata2039. The process starts by defining an S3 configuration, which includes the bucket's ARN and inclusion prefixes. The inclusion prefixes are used to specify which files in the bucket should be included; this can range from a specific file name to a wildcard '*' to include all files.

Once the S3 configuration is set, it's incorporated into a larger data source configuration, which is then used to create the data source through the Amazon Bedrock API. The API call requires details like the name and description of the data source, as well as the ID of the knowledge base to which this data source is to be added.

After the data source is successfully created and linked to the knowledge base, the next step is to initiate a synchronization process. This process involves Amazon Bedrock scanning the specified S3 bucket, based on the inclusion prefixes, and ingesting the data into the knowledge base. During this synchronization, the data is processed, and potentially, embeddings are created based on the configured embedding model. This makes the data searchable and retrievable through the knowledge base.

Once synchronization is complete, the content from the S3 bucket is available in the knowledge base. You can then perform queries against this knowledge base to retrieve relevant information based on your search criteria. This integration allows for a seamless connection between your stored data in S3 and the powerful search and retrieval capabilities offered by Amazon Bedrock, making it a robust solution for managing and accessing large volumes of data efficiently.

## Retrieving content and evaluating using OpenSource Frameworks

In [22]:
import requests
import boto3
import json
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest

# Replace these variables with your actual values
knowledge_base_id = ''
api_endpoint = 'https://bedrock-agent-runtime.us-east-1.amazonaws.com'
query_text = 'health spending'
number_of_results = 3

# Construct the URL
url = f"{api_endpoint}/knowledgebases/{knowledge_base_id}/retrieve"

# Construct the payload
payload = {
    "retrievalConfiguration": {
        "vectorSearchConfiguration": {
            "numberOfResults": number_of_results
        }
    },
    "retrievalQuery": {
        "text": query_text
    }
}

# Create a request object
request = AWSRequest(method="POST", url=url, data=json.dumps(payload), headers={'Content-Type': 'application/json'})

# Use Boto3 to get the current session's credentials
session = boto3.Session()
credentials = session.get_credentials()

# Create a SigV4Auth object with 'bedrock' as the service name
auth = SigV4Auth(credentials, 'bedrock', session.region_name)

# Sign the request
auth.add_auth(request)

# Make the POST request with the signed headers
response = requests.post(url, headers=dict(request.headers), data=request.body)

# Check if the request was successful
if response.status_code == 200:
    # Process the response
    results = response.json()
    print("Retrieval Results:", results)
else:
    print("Error:", response.status_code, response.text)
